# Testing Flask Editable Widgets

In [33]:
# !pip install forgebox
# !rm sample.db

## Create a fake database

In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine as ce

con = ce("sqlite:///sample.db")

sample_df = pd.DataFrame(dict(name=["Darrow","Virginia","Sevro",]*20,
                              house =["Andromedus","Augustus","Barca"]*20,
                              age=[20,18,17]*20))

sample_df.to_sql("sample_table",index_label="id",
                 index=True,
                 con = con, method='multi',
                 if_exists="replace")

In [3]:
!sqlite3 sample.db .tables

sample_table


In [5]:
!sqlite3 sample.db "SELECT * FROM sample_table LIMIT 10"

0|Darrow|Andromedus|20
1|Virginia|Augustus|18
2|Sevro|Barca|17
3|Darrow|Andromedus|20
4|Virginia|Augustus|18
5|Sevro|Barca|17
6|Darrow|Andromedus|20
7|Virginia|Augustus|18
8|Sevro|Barca|17
9|Darrow|Andromedus|20


## Create a editable widget

In [6]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

In [8]:
from forgebox.editable import Editable

Editable("task1", # route/task name
         app, # flask app to wrap around
         table_name="sample_table", # target table name
         id_col="id", # unique column
         con = con,
         log_con=con)

In [9]:
app.run(port = 4242)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4242/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jun/2020 18:58:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 18:58:57] "GET /task1 HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 18:58:57] "GET /task1/df_api?page=0&where= HTTP/1.1" 200 -


SELECT * FROM sample_table 
        ORDER BY id ASC LIMIT 0,20
        


127.0.0.1 - - [21/Jun/2020 18:59:09] "GET /task1/df_api?page=1&where= HTTP/1.1" 200 -


SELECT * FROM sample_table 
        ORDER BY id ASC LIMIT 1,20
        


...

SELECT * FROM sample_table WHERE house='Barca' 
        ORDER BY id ASC LIMIT 0,20
        


127.0.0.1 - - [21/Jun/2020 18:59:45] "GET /task1/df_api?page=0&where=age%3E17 HTTP/1.1" 200 -


SELECT * FROM sample_table WHERE age>17 
        ORDER BY id ASC LIMIT 0,20
        


127.0.0.1 - - [21/Jun/2020 19:00:18] "POST /task1/save_api HTTP/1.1" 200 -


  idx    col valtype    original  changed         ip    table_name  \
0   0   name     str      Darrow   Reaper  127.0.0.1  sample_table   
1   0  house     str  Andromedus  Ringing  127.0.0.1  sample_table   
2   3    age     int          20       23  127.0.0.1  sample_table   
3   4   name     str    Virginia  Mustang  127.0.0.1  sample_table   

                          ts  \
0 2020-06-21 19:00:18.015897   
1 2020-06-21 19:00:18.021918   
2 2020-06-21 19:00:18.025394   
3 2020-06-21 19:00:18.027062   

                                                 sql  
0  UPDATE sample_table\n            SET name='Rea...  
1  UPDATE sample_table\n            SET house='Ri...  
2  UPDATE sample_table\n            SET age=23 WH...  
3  UPDATE sample_table\n            SET name='Mus...  
SELECT * FROM sample_table WHERE age>17 
        ORDER BY id ASC LIMIT 0,20
        


In [10]:
from forgebox.df import PandasDisplay
with PandasDisplay(max_colwidth = 0,max_rows=100):
    display(pd.read_sql("editable_log",con=con))

,idx,col,valtype,original,changed,ip,table_name,ts,sql
0,0,name,str,Darrow,Reaper,127.0.0.1,sample_table,2020-06-21 19:00:18.015897,UPDATE sample_table\n SET name='Reaper' WHERE id=0\n
1,0,house,str,Andromedus,Ringing,127.0.0.1,sample_table,2020-06-21 19:00:18.021918,UPDATE sample_table\n SET house='Ringing' WHERE id=0\n
2,3,age,int,20,23,127.0.0.1,sample_table,2020-06-21 19:00:18.025394,UPDATE sample_table\n SET age=23 WHERE id=3\n
3,4,name,str,Virginia,Mustang,127.0.0.1,sample_table,2020-06-21 19:00:18.027062,UPDATE sample_table\n SET name='Mustang' WHERE id=4\n
